# Create Stores Content

In [1]:
import os, sys, random
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import engine.models as GM

import engine.logic.content as CT

In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Get Guild

In [8]:
userMd = get_user_model().objects.filter(user_name='Admin')[0]
guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Selected=True)
PD.DataFrame([guildMd.__dict__]).drop(['id', '_state'], axis=1, errors='ignore')

,UserFK_id,Name,LastPlayed,CreateDate,Selected,ThroneLevel,CampaignWorld,TotalPower,MaxThieves,StorageGold,StorageGems,StorageWood,StorageStone,StorageIron,VaultGold,VaultGems,VaultWood,VaultStone,VaultIron,DungeonCheck
0,1,433,2024-08-03,2024-08-03,True,1,1,417,6,3320,0,890,0,0,443,202,890,0,0,2024-08-04


In [9]:
# unlocked blueprints report

unlockThief = EM.UnlockableThief.objects.filter(Stars__gt=1).values()

PD.DataFrame(unlockThief)#.drop(['id', '_state'], axis=1, errors='ignore')

,id,ResourceId,Class,Stars,UnlockThrone,StoreCost,StartTrait,RandomTraits
0,160,thief-agi-s2,Burglar,2,1,3350,agi 4,2
1,161,thief-cun-s2,Scoundrel,2,1,3350,cun 4,2
2,162,thief-mig-s2,Ruffian,2,1,3350,mig 4,2
3,163,thief-agi-s3,Burglar,3,4,4500,agi 5,4
4,164,thief-cun-s3,Scoundrel,3,4,4500,cun 5,4
5,165,thief-mig-s3,Ruffian,3,4,4500,mig 5,4
6,166,thief-agi-s4,Burglar,4,7,5850,agi 6,6
7,167,thief-cun-s4,Scoundrel,4,7,5850,cun 6,6
8,168,thief-mig-s4,Ruffian,4,7,5850,mig 6,6


In [13]:
for th in unlockThief:
    th['IconCode'] = f"class-{th['Class'].lower()}-s{th['Stars']}"
    checkUnlock = GM.ThiefUnlocked.objects.GetOrNone(GuildFK=guildMd, ThiefFK__ResourceId=th['ResourceId'])
    th['Unlocked'] = True if checkUnlock else False
    
PD.DataFrame(unlockThief)

,id,ResourceId,Class,Stars,UnlockThrone,StoreCost,StartTrait,RandomTraits,IconCode,Unlocked
0,160,thief-agi-s2,Burglar,2,1,3350,agi 4,2,class-burglar-s2,False
1,161,thief-cun-s2,Scoundrel,2,1,3350,cun 4,2,class-scoundrel-s2,True
2,162,thief-mig-s2,Ruffian,2,1,3350,mig 4,2,class-ruffian-s2,False
3,163,thief-agi-s3,Burglar,3,4,4500,agi 5,4,class-burglar-s3,False
4,164,thief-cun-s3,Scoundrel,3,4,4500,cun 5,4,class-scoundrel-s3,False
5,165,thief-mig-s3,Ruffian,3,4,4500,mig 5,4,class-ruffian-s3,False
6,166,thief-agi-s4,Burglar,4,7,5850,agi 6,6,class-burglar-s4,False
7,167,thief-cun-s4,Scoundrel,4,7,5850,cun 6,6,class-scoundrel-s4,False
8,168,thief-mig-s4,Ruffian,4,7,5850,mig 6,6,class-ruffian-s4,False
